#Rarefaction, beta-diversity analysis and alpha-diversity calculations

In [ ]:
#Import packages
library(phyloseq)
library(vegan)
library(ggplot2)
library(pspearman)
library(OneR)
library(plyr)
library(dplyr)

In [ ]:
#import phyloseq object with 16s count data and metadata
data<-readRDS('statins_16S.rds')
data

In [ ]:
#Rarefy the dataset to even depth
print(min(sample_sums(data)))
rarefied_data=rarefy_even_depth(data, sample.size = min(sample_sums(data)),
  rngseed = 321, replace = FALSE, trimOTUs = TRUE, verbose = TRUE)
#save the rarefied phyloseq object file
saveRDS(rarefied_data,'rarefied_16s.rds')

In [ ]:
rarefied_data<-readRDS('rarefied_16s.rds')
rarefied_data

In [ ]:
rarefied_phylum<-tax_glom(rarefied_data,"Phylum")
saveRDS(rarefied_phylum,'rarefied_genus_16S.rds')

In [ ]:
rarefied_genus<-tax_glom(rarefied_data,"Genus")

In [ ]:
saveRDS(rarefied_genus,'rarefied_genus_16S.rds')

In [ ]:
rarefied_genus<-readRDS('rarefied_genus_16S.rds')

In [ ]:
#Perform beta-diversity analysis on the bray-curtis dissimilarity matrix
set.seed(123)
metadata <- as(sample_data(rarefied_genus), "data.frame")
#calculate distance matrix
bray<-distance(rarefied_genus, method="bray")
#test for association between interindividual gut microbiome variation and statin use, statin*HMG interaction
adonis(bray~ Group*HMG,
       data = metadata,permutations = 3000)
adonis(bray ~ vendor_microbiome+BMI+sex+age+Group*HMG,
       data = metadata,permutations = 3000)

In [ ]:
#Perform beta-diversity analysis on the Weighted Unifrac dissimilarity matrix
set.seed(123)
wunifrac<-distance(rarefied_genus, method="wunifrac")
#test for association between interindividual gut microbiome variation and statin use
adonis(wunifrac~ Group*HMG,
       data = metadata,permutations = 3000)
adonis(wunifrac~ vendor_microbiome+BMI+sex+age+Group*HMG,
       data = metadata,permutations = 3000)

In [ ]:
rarefied_genotek<-readRDS('rarefied_16s.rds')
#calculate alpha-diversity measures rarefied using ASV count data
richness_df<-estimate_richness(rarefied_genotek,measures=c('Observed','Shannon'))

In [ ]:
#add participant id to the df
richness_df$public_client_id<-sample_data(rarefied_genotek)$public_client_id

In [ ]:
#export alpha-diversity dataframe for further statistical analysis
write.csv(richness_df,'richness_df.csv')